In [1]:
import time

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus

from random_lp.lp_random_gen import create_models

DIR = 'TEST_DATA' + "/" + time.strftime("%d_%m_%Y") + "/DENSE/" # 12.03.2021
Q_SEED = 10598 # as used in most issues
aqua_globals.random_seed = Q_SEED

In [2]:
# select linear program to solve
qps = create_models(DIR)
qp = qps['test_8']
qp.to_docplex().prettyprint()

// This file has been generated by DOcplex
// model name is: test_8
// single vars section
dvar bool x0_0;
dvar bool x0_2;
dvar bool x0_3;
dvar bool x0_4;
dvar bool x0_6;
dvar bool x0_1;
dvar bool x0_7;
dvar bool x0_5;

minimize
 - 2 x0_0 + 2 x0_2 + x0_3 - x0_4 - x0_6;
 
subject to {
 A0_leb0:
  2 x0_0 - 3 x0_2 - x0_4 + x0_6 - x0_1 - 3 x0_7 <= 1;
 A0_leb1:
  - 2 x0_0 - x0_2 - 2 x0_3 + 2 x0_4 - 2 x0_6 - 2 x0_1 + 3 x0_5 <= 3;
 A0_leb2:
  - 2 x0_2 - 3 x0_3 - 2 x0_4 - 3 x0_6 + 3 x0_7 - 2 x0_5 <= -6;
 A0_leb3:
  - 3 x0_0 + 3 x0_3 + x0_4 - 2 x0_6 + 2 x0_1 + 2 x0_7 + x0_5 <= 2;

}


In [3]:
# init classical Optimizers
optimizer = COBYLA()
cplex = CplexOptimizer()

In [4]:
# solve qps with Minimum Eigen Optimizer QAOA
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,
                                   seed_simulator=Q_SEED,
                                   seed_transpiler=Q_SEED,
                                   shots=shots)

# QAOA
qaoa_mes = QAOA(quantum_instance=quantum_instance, optimizer=optimizer)
qaoa = MinimumEigenOptimizer(qaoa_mes)

In [5]:
cplex.solve(qp)

optimal function value: -4.0
optimal value: [ 1. -0. -0.  1.  1.  1. -0.  1.]
status: SUCCESS

In [6]:
print("number of qubits: ",qp.qubo.get_num_vars())

number of qubits:  22


In [7]:
res = qaoa.solve(qp)
res

constraint A0_leb3 is infeasible due to substitution


optimal function value: -2.0
optimal value: [1. 1. 0. 1. 1. 1. 0. 1.]
status: SUCCESS

In [8]:
qp.is_feasible(res.x)

True